In [81]:
from pyopenms import *

In [82]:
mzML_files = ["Example_data/Actino_1.mzML", "Example_data/Actino_2.mzML"]
feature_maps = []

In [ ]:
for mzML_file in mzML_files:
    exp = MSExperiment()
    MzMLFile().load(mzML_file, exp)
    exp.sortSpectra(True)

    print("Mass Trace Detection: ", mzML_file)
    mtd = MassTraceDetection()
    mtd_par = mtd.getDefaults()
    mtd_par.setValue("mass_error_ppm", 10.0) 
    mtd_par.setValue("noise_threshold_int", 1.0e04)
    mtd.setParameters(mtd_par)
    mass_traces = []
    mtd.run(exp, mass_traces, 0)

    print("Elution Peak Detection: ", mzML_file)
    epd = ElutionPeakDetection()
    epd_par = epd.getDefaults()
    epd_par.setValue("width_filtering", "fixed")
    epd.setParameters(epd_par)
    mass_traces_final = []
    epd.detectPeaks(mass_traces, mass_traces_final)

    ffm = FeatureFindingMetabo()
    ffm_par = ffm.getDefaults() 
    ffm_par.setValue("isotope_filtering_model", "none")
    ffm_par.setValue("remove_single_traces", "true")
    ffm_par.setValue("mz_scoring_by_elements", "false")
    ffm_par.setValue("report_convex_hulls", "true")
    ffm.setParameters(ffm_par)
    feature_map = FeatureMap()
    feat_chrom = []
    ffm.run(mass_traces_final, feature_map, feat_chrom)

    feature_map.setUniqueIds()
    feature_map.setPrimaryMSRunPath([mzML_file.encode()])
    feature_maps.append(feature_map)

In [ ]:
feature_maps[0].get_df(export_peptide_identifications=False)

In [85]:
aligner = MapAlignmentAlgorithmPoseClustering()
aligner_par= aligner.getDefaults()
aligner_par.setValue("max_num_peaks_considered", -1)
aligner_par.setValue("superimposer:mz_pair_max_distance", 0.05)
aligner_par.setValue("pairfinder:distance_MZ:max_difference", 10.0)
aligner_par.setValue("pairfinder:distance_MZ:unit", "ppm")
aligner.setParameters(aligner_par)

ref_index = feature_maps.index(sorted(feature_maps, key=lambda x:x.size())[-1])
aligner.setReference(feature_maps[ref_index])

for feature_map in feature_maps[:ref_index] + feature_maps[ref_index+1:]:
    trafo = TransformationDescription()
    aligner.align(feature_map, trafo)
    transformer = MapAlignmentTransformer()
    transformer.transformRetentionTimes(feature_map, trafo, True)

In [ ]:
feature_grouper = FeatureGroupingAlgorithmKD()

consensus_map = ConsensusMap()
file_descriptions = consensus_map.getColumnHeaders()

for i, feature_map in enumerate(feature_maps):
    file_description = file_descriptions.get(i, ColumnHeader())
    file_description.filename = os.path.basename(feature_map.getMetaValue("spectra_data")[0].decode())
    file_description.size = feature_map.size()
    file_descriptions[i] = file_description

feature_grouper.group(feature_maps, consensus_map)
consensus_map.setColumnHeaders(file_descriptions)

In [ ]:
result = consensus_map.get_df()
result

In [88]:
ConsensusXMLFile().store("results/ConsensusFeatures.consensusXML", consensus_map)

result.to_csv("results/FeatureMatrix.tsv", sep = '\t', index = False)